In [8]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('default')
  


import warnings
warnings.filterwarnings('ignore')

In [25]:
laliga2122 = pd.read_excel('LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('LaLiga16 17.xlsx', )

In [2]:
seg2122 =  pd.read_excel('LaLiga123 21 22.xlsx', )
seg2021 =  pd.read_excel('LaLiga123 20 21.xlsx', )
seg1920 =  pd.read_excel('LaLiga123 19 20.xlsx', )
seg1819 =  pd.read_excel('LaLiga123 18 19.xlsx', )
seg1718 =  pd.read_excel('LaLiga123 17 18.xlsx', )
seg1617 =  pd.read_excel('LaLiga123 16 17.xlsx', )


### 1

Si se hace una agrupación de los equipos que alguna vez estuvieron en primera división con K=4 grupos, y considerando solamente las siguientes variables para ello: puntos totales y asistencia de local, ¿cuál de las siguientes alternativas indica la cantidad de equipos que son clasificados junto al Sevilla (cuenta sin incluir al Sevilla)?  (use random_seed=55)  

R: 5

In [11]:
laliga2122

,Wk,Day,Date,Time,Local,xG,Score,xG.1,Visitante,Attendance,Venue,Referee,Informe del partido,Notes
0,1.0,Vie,2021-08-13,21:00 (15:00),Valencia,1.3,1–0,1.2,Getafe,9.868,Estadio de Mestalla,Jesús Gil,Informe del partido,NaN
1,1.0,Sáb,2021-08-14,19:30 (13:30),Cádiz,0.4,1–1,1.0,Levante,6.941,Estadio Nuevo Mirandilla,Santiago Jaime,Informe del partido,NaN
2,1.0,Sáb,2021-08-14,19:30 (13:30),Mallorca,0.4,1–1,0.9,Betis,6.279,Iberostar Estadi,Antonio Matéu Lahoz,Informe del partido,NaN
3,1.0,Sáb,2021-08-14,22:00 (16:00),Osasuna,0.8,0–0,0.7,Espanyol,6.715,Estadio El Sadar,Miguel Ángel Ortiz Arias,Informe del partido,NaN
4,1.0,Sáb,2021-08-14,22:00 (16:00),Alavés,1.7,1–4,1.8,Real Madrid,3.968,Estadio de Mendizorroza,César Soto,Informe del partido,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,38.0,Dom,2022-05-22,20:00 (14:00),Alavés,0.4,0–1,1.9,Cádiz,11.138,Estadio de Mendizorroza,José Sánchez,Informe del partido,NaN
420,38.0,Dom,2022-05-22,20:00 (14:00),Granada,2.0,0–0,0.4,Espanyol,17.951,Estadio Nuevo Los Cármenes,Alejandro Hernández,Informe del partido,NaN
421,38.0,Dom,2022-05-22,22:00 (16:00),Real Sociedad,2.3,1–2,0.8,Atlético Madrid,23.586,Estadio Municipal de Anoeta,Jesús Gil,Informe del partido,NaN
422,38.0,Dom,2022-05-22,22:00 (16:00),Barcelona,0.7,0–2,0.7,Villarreal,54.850,Camp Nou,José Luis Munuera,Informe del partido,NaN


In [19]:
import pandas as pd
from sklearn.cluster import KMeans

all_seasons = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617])

# Convertir la columna 'Score' a string y manejar valores nulos
all_seasons['Score'] = all_seasons['Score'].astype(str)

# Extraer la columna de Local, Puntos y Asistencia
def calculate_points(score):
    try:
        home_goals, away_goals = map(int, score.split('–'))
        if home_goals > away_goals:
            return 3
        elif home_goals == away_goals:
            return 1
        else:
            return 0
    except:
        return 0

all_seasons['Puntos'] = all_seasons['Score'].apply(calculate_points)

# Agrupar por equipo y sumar puntos y asistencia
teams_points = all_seasons.groupby('Local').agg({'Puntos': 'sum', 'Attendance': 'sum'}).reset_index()

# Aplicar KMeans clustering
X = teams_points[['Puntos', 'Attendance']]
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(X)

# Determinar el cluster del Sevilla
sevilla_cluster = teams_points[teams_points['Local'] == 'Sevilla']['Cluster'].values[0]

# Contar equipos en el mismo cluster que el Sevilla, excluyendo al Sevilla mismo
teams_in_same_cluster = teams_points[teams_points['Cluster'] == sevilla_cluster]
count_teams = len(teams_in_same_cluster) - 1

print(f'Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): {count_teams}')


Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): 3


In [21]:
import pandas as pd
from sklearn.cluster import KMeans

# Concatenar todos los DataFrames en uno
all_seasons = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617])

# Convertir la columna 'Score' a string y manejar valores nulos
all_seasons['Score'] = all_seasons['Score'].astype(str)

# Función para calcular los puntos
def calculate_points(score):
    try:
        home_goals, away_goals = map(int, score.split('–'))
        if home_goals > away_goals:
            return 3
        elif home_goals == away_goals:
            return 1
        else:
            return 0
    except:
        return 0

# Aplicar la función a la columna 'Score'
all_seasons['Puntos'] = all_seasons['Score'].apply(calculate_points)

# Agrupar por equipo y sumar puntos y asistencia
teams_points = all_seasons.groupby('Local').agg({'Puntos': 'sum', 'Attendance': 'sum'}).reset_index()

# Aplicar K-means clustering
X = teams_points[['Puntos', 'Attendance']]
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(X)

# Determinar el cluster del Sevilla
sevilla_cluster = teams_points[teams_points['Local'] == 'Sevilla']['Cluster'].values[0]

# Contar equipos en el mismo cluster que el Sevilla, excluyendo al Sevilla mismo
teams_in_same_cluster = teams_points[teams_points['Cluster'] == sevilla_cluster]
count_teams = len(teams_in_same_cluster) - 1

print(f'Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): {count_teams}')


Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): 3


In [26]:
all_seasons=[]

In [27]:
import pandas as pd
from sklearn.cluster import KMeans


# Unir los DataFrames
all_seasons = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617])

# Convertir la columna 'Score' a string y manejar valores nulos
all_seasons['Score'] = all_seasons['Score'].astype(str)

# Función para calcular los puntos para local y visitante
def calculate_points(row):
    try:
        home_goals, away_goals = map(int, row['Score'].split('–'))
        if home_goals > away_goals:
            return (3, 0)  # Local gana, Visitante pierde
        elif home_goals == away_goals:
            return (1, 1)  # Empate
        else:
            return (0, 3)  # Local pierde, Visitante gana
    except:
        return (0, 0)  # En caso de error

# Aplicar la función a cada fila para obtener los puntos de local y visitante
all_seasons[['PuntosLocal', 'PuntosVisitante']] = all_seasons.apply(calculate_points, axis=1, result_type='expand')

# Sumar los puntos y asistencia por equipo
points_local = all_seasons.groupby('Local').agg({'PuntosLocal': 'sum', 'Attendance': 'sum'}).reset_index()
points_visitante = all_seasons.groupby('Visitante').agg({'PuntosVisitante': 'sum'}).reset_index()

# Renombrar columnas para evitar conflictos al fusionar
points_visitante.rename(columns={'Visitante': 'Equipo', 'PuntosVisitante': 'Puntos'}, inplace=True)
points_local.rename(columns={'Local': 'Equipo', 'PuntosLocal': 'Puntos'}, inplace=True)

# Unir los puntos de local y visitante
teams_points = pd.merge(points_local, points_visitante, on='Equipo', how='outer').fillna(0)
teams_points['Puntos'] = teams_points['Puntos_x'] + teams_points['Puntos_y']
teams_points = teams_points[['Equipo', 'Puntos', 'Attendance']]

# Aplicar KMeans clustering
X = teams_points[['Puntos', 'Attendance']]
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(X)

# Determinar el cluster del Sevilla
sevilla_cluster = teams_points[teams_points['Equipo'] == 'Sevilla']['Cluster'].values[0]

# Contar equipos en el mismo cluster que el Sevilla, excluyendo al Sevilla mismo
teams_in_same_cluster = teams_points[teams_points['Cluster'] == sevilla_cluster]
count_teams = len(teams_in_same_cluster) - 1

print(f'Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): {count_teams}')


Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): 3


In [30]:
import pandas as pd
from sklearn.cluster import KMeans



# Unir los DataFrames
all_seasons = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617])

# Convertir la columna 'Score' a string y manejar valores nulos
all_seasons['Score'] = all_seasons['Score'].astype(str)

# Función para calcular los puntos para local y visitante
def calculate_points(row):
    try:
        home_goals, away_goals = map(int, row['Score'].split('–'))
        if home_goals > away_goals:
            return (3, 0)  # Local gana, Visitante pierde
        elif home_goals == away_goals:
            return (1, 1)  # Empate
        else:
            return (0, 3)  # Local pierde, Visitante gana
    except:
        return (0, 0)  # En caso de error

# Aplicar la función a cada fila para obtener los puntos de local y visitante
all_seasons[['PuntosLocal', 'PuntosVisitante']] = all_seasons.apply(calculate_points, axis=1, result_type='expand')

# Sumar los puntos y asistencia por equipo y temporada
points_local = all_seasons.groupby(['Local']).agg({'PuntosLocal': 'sum', 'Attendance': 'sum'}).reset_index()
points_visitante = all_seasons.groupby(['Visitante']).agg({'PuntosVisitante': 'sum'}).reset_index()

# Renombrar columnas para evitar conflictos al fusionar
points_visitante.rename(columns={'Visitante': 'Equipo', 'PuntosVisitante': 'PuntosVisitante'}, inplace=True)
points_local.rename(columns={'Local': 'Equipo', 'PuntosLocal': 'PuntosLocal'}, inplace=True)

# Unir los puntos de local y visitante por equipo
season_points = pd.merge(points_local, points_visitante, on='Equipo', how='outer').fillna(0)
season_points['PuntosTotales'] = season_points['PuntosLocal'] + season_points['PuntosVisitante']

# Sumar los puntos totales y asistencia por equipo en todas las temporadas
teams_points = season_points.groupby('Equipo').agg({'PuntosTotales': 'sum', 'Attendance': 'sum'}).reset_index()

# Aplicar KMeans clustering
X = teams_points[['PuntosTotales', 'Attendance']]
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(X)

# Determinar el cluster del Sevilla
sevilla_cluster = teams_points[teams_points['Equipo'] == 'Sevilla']['Cluster'].values[0]

# Contar equipos en el mismo cluster que el Sevilla, excluyendo al Sevilla mismo
teams_in_same_cluster = teams_points[(teams_points['Cluster'] == sevilla_cluster) & (teams_points['Equipo'] != 'Sevilla')]
count_teams = len(teams_in_same_cluster)

print(f'Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): {count_teams}')



Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): 3


In [40]:
import pandas as pd
from sklearn.cluster import KMeans

laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all'))
laliga2 = laliga.loc[:,['Wk','Date','Local','xG','Score','xG.1','Visitante', 'Attendance']]

# Convertir la columna 'Score' a string y manejar valores nulos
all_seasons['Score'] = all_seasons['Score'].astype(str)
all_seasons['Attendance']=all_seasons['Attendance']

# Función para calcular los puntos para local y visitante
def calculate_points(row):
    try:
        home_goals, away_goals = map(int, row['Score'].split('–'))
        if home_goals > away_goals:
            return (3, 0)  # Local gana, Visitante pierde
        elif home_goals == away_goals:
            return (1, 1)  # Empate
        else:
            return (0, 3)  # Local pierde, Visitante gana
    except:
        return (0, 0)  # En caso de error

# Aplicar la función a cada fila para obtener los puntos de local y visitante
all_seasons[['PuntosLocal', 'PuntosVisitante']] = all_seasons.apply(calculate_points, axis=1, result_type='expand')

# Agrupar por equipo y temporada hasta la última fecha
points_local = all_seasons.groupby(['Local', 'Wk']).agg({'PuntosLocal': 'sum', 'Attendance': 'sum'}).reset_index()
points_visitante = all_seasons.groupby(['Visitante', 'Wk']).agg({'PuntosVisitante': 'sum'}).reset_index()

# Renombrar columnas para evitar conflictos al fusionar
points_visitante.rename(columns={'Visitante': 'Equipo', 'PuntosVisitante': 'PuntosVisitante'}, inplace=True)
points_local.rename(columns={'Local': 'Equipo', 'PuntosLocal': 'PuntosLocal', 'Attendance': 'Attendance'}, inplace=True)

# Unir los puntos de local y visitante por equipo y temporada
season_points = pd.merge(points_local, points_visitante, on=['Equipo', 'Wk'], how='outer').fillna(0)
season_points['PuntosTotales'] = season_points['PuntosLocal'] + season_points['PuntosVisitante']

# Sumar los puntos totales y asistencia por equipo en todas las temporadas
teams_points = season_points.groupby('Equipo').agg({'PuntosTotales': 'sum', 'Attendance': 'sum'}).reset_index()

# Aplicar KMeans clustering
X = teams_points[['PuntosTotales', 'Attendance']]
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(X)

# Determinar el cluster del Sevilla
sevilla_cluster = teams_points[teams_points['Equipo'] == 'Sevilla']['Cluster'].values[0]

# Contar equipos en el mismo cluster que el Sevilla, excluyendo al Sevilla mismo
teams_in_same_cluster = teams_points[(teams_points['Cluster'] == sevilla_cluster) & (teams_points['Equipo'] != 'Sevilla')]
count_teams = len(teams_in_same_cluster)

print(f'Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): {count_teams}')


Cantidad de equipos en el mismo grupo que el Sevilla (sin incluir al Sevilla): 3


In [37]:
import pandas as pd
from sklearn.cluster import KMeans

# Leer los archivos Excel y ajustar la columna de asistencia
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2122['Attendance'] *= 1000

laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga2021['Attendance'] *= 1000

laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1920['Attendance'] *= 1000

laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1819['Attendance'] *= 1000

laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1718['Attendance'] *= 1000

laliga1617 = pd.read_excel('LaLiga16 17.xlsx')
laliga1617['Attendance'] *= 1000

# Unir los DataFrames usando append y seleccionar columnas relevantes
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')) \
                   .append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')) \
                   .append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all'))

# Calcular los puntos totales por equipo considerando la última fecha (wk)
laliga['Puntos'] = laliga['Score'].apply(lambda x: int(x.split('–')[0]) * 3 if x.split('–')[0] > x.split('–')[1] else (int(x.split('–')[0]) if x.split('–')[0] == x.split('–')[1] else 0))
teams_points = laliga.groupby('Local').agg({'Puntos': 'sum', 'Attendance': 'sum'}).reset_index()

# Aplicar KMeans clustering para agrupar equipos
kmeans = KMeans(n_clusters=4, random_state=55)
teams_points['Cluster'] = kmeans.fit_predict(teams_points[['Puntos', 'Attendance']])

# Identificar el cluster del equipo Sevilla
sevilla_cluster = teams_points.loc[teams_points['Local'] == 'Sevilla', 'Cluster'].values[0]

# Contar la cantidad de equipos clasificados en el mismo cluster que Sevilla (sin incluir a Sevilla)
num_teams_same_cluster = teams_points[teams_points['Cluster'] == sevilla_cluster]['Local'].nunique() - 1

print(f"La cantidad de equipos clasificados junto al Sevilla (sin incluir al Sevilla) es: {num_teams_same_cluster}")


La cantidad de equipos clasificados junto al Sevilla (sin incluir al Sevilla) es: 3


In [54]:
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all')).loc[:,
['Wk','Date','Local','Score','Visitante', 'Attendance']]
laliga['Attendance']= laliga['Attendance']*1000
laliga['Torneo'] = 'Primera división'

In [55]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Total goles partido']= laliga['Goles Local']+laliga['Goles Visitante']


In [56]:
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)

In [60]:
import pandas as pd

def puntos(equipo1,equipo2,data):
    usos = data[((data['Local']==equipo1)&(data['Visitante']==equipo2))|((data['Local']==equipo2)&(data['Visitante']==equipo1))]
    puntos = 0
    for i in range(len(usos)):
        ganador = str(usos.iloc[i]['Ganador'])
        if ganador == equipo1:
            puntos += 3
        if ganador == 'Empate':
            puntos +=1
    return puntos
# Aplicar la función puntos() para calcular los puntos totales de todos los equipos
equipos = laliga['Local'].unique()
puntos_totales = {}
for equipo in equipos:
    puntos_totales[equipo] = sum(puntos(equipo, otro_equipo, laliga) for otro_equipo in equipos if otro_equipo != equipo)

# Obtener la cantidad de equipos clasificados junto al Sevilla (sin incluir al Sevilla)
sevilla = 'Sevilla'
equipos_clasificados = sum(1 for equipo, puntos in puntos_totales.items() if equipo != sevilla and puntos > puntos_totales[sevilla])

# Imprimir la cantidad de equipos clasificados junto al Sevilla
print(f"La cantidad de equipos clasificados junto al Sevilla es: {equipos_clasificados}")


La cantidad de equipos clasificados junto al Sevilla es: 3


In [59]:
agrup_equip = laliga[laliga['Ganador']!= 'Empate'].groupby(['Ganador'])['Wk'].count().reset_index()[['Ganador']]
agrup_equip

,Ganador
0,Alavés
1,Athletic Club
2,Atlético Madrid
3,Barcelona
4,Betis
5,Celta Vigo
6,Cádiz
7,Eibar
8,Elche
9,Espanyol


In [65]:
import pandas as pd

# Unir todos los DataFrames de temporadas anteriores de La Liga
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1617 = pd.read_excel('LaLiga16 17.xlsx')

# Concatenar todos los DataFrames
laliga = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617], ignore_index=True)

# Ajustar columnas y datos según sea necesario
laliga['Attendance'] = laliga['Attendance'] * 1000
laliga['Torneo'] = 'Primera División'
laliga['Score'] = laliga['Score'].astype(str)

def golesvisita(marcador):
    goles = marcador.split('–')
    if len(goles) == 2:  # Verificar si hay exactamente dos valores
        return int(goles[1])
    else:
        return 0  # O devolver 0, dependiendo de la lógica de tu aplicación

def goleslocal(marcador):
    goles = marcador.split('–')
    if len(goles) == 2:  # Verificar si hay exactamente dos valores
        return int(goles[0])
    else:
        return 0  # O devolver 0, dependiendo de la lógica de tu aplicación

def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if len(goles) == 2:
        if int(goles[0]) > int(goles[1]):
            return local
        elif int(goles[0]) < int(goles[1]):
            return visita
        else:
            return 'Empate'
    else:
        return 'Empate'  # Manejar el caso de empate o marcador no válido

laliga['Goles Local'] = laliga['Score'].apply(goleslocal)
laliga['Goles Visitante'] = laliga['Score'].apply(golesvisita)
laliga['Ganador'] = laliga.apply(lambda x: resultado_equipo(x['Score'], x['Local'], x['Visitante']), axis=1)

# Función para calcular los puntos de un equipo en base a los partidos jugados
def puntos(equipo1, equipo2, data):
    usos = data[((data['Local'] == equipo1) & (data['Visitante'] == equipo2)) | ((data['Local'] == equipo2) & (data['Visitante'] == equipo1))]
    puntos = 0
    for i in range(len(usos)):
        ganador = str(usos.iloc[i]['Ganador'])
        if ganador == equipo1:
            puntos += 3
        elif ganador == 'Empate':
            puntos += 1
    return puntos

# Limpiar valores NaN en la columna 'Local'
laliga['Local'] = laliga['Local'].fillna('').astype(str)

# Obtener todos los equipos que alguna vez estuvieron en Primera División
equipos = sorted(laliga['Local'].unique().tolist())

# Calcular los puntos totales para cada equipo
puntos_totales = {}
for equipo in equipos:
    puntos_totales[equipo] = sum(puntos(equipo, otro_equipo, laliga) for otro_equipo in equipos if otro_equipo != equipo)

# Determinar cuántos equipos clasifican junto al Sevilla (sin incluir al Sevilla)
sevilla = 'Sevilla'
equipos_clasificados = sum(1 for equipo, puntos in puntos_totales.items() if equipo != sevilla and puntos > puntos_totales[sevilla])

# Imprimir la cantidad de equipos clasificados junto al Sevilla
print(f"La cantidad de equipos clasificados junto al Sevilla es: {equipos_clasificados}")


La cantidad de equipos clasificados junto al Sevilla es: 3


In [71]:
# Calcular la cantidad de equipos en cada clúster
teams_per_cluster = teams_points.groupby('Cluster')['Local'].count()

# Obtener el clúster con menos equipos
min_teams_cluster = teams_per_cluster.idxmin()
min_teams_count = teams_per_cluster[min_teams_cluster]

# Obtener el clúster con más equipos
max_teams_cluster = teams_per_cluster.idxmax()
max_teams_count = teams_per_cluster[max_teams_cluster]

print(f"Cantidad de equipos en el clúster con menos equipos: {min_teams_count}")
print(f"Cantidad de equipos en el clúster con más equipos: {max_teams_count}")


KeyError: 'Column not found: Local'

### 2

Pregunta 2

Utilizando el resultado de la Pregunta 1 (la etiqueta de los equipos), ¿cuál de las siguientes alternativas corresponde a la cantidad de equipos que hay en el clúster que quedó con menos y con más equipos respectivamente? 

R: 3 y 11

### 3

Pregunta 3

Si se hace una regresión multivariada para predecir la asistencia al estadio en función de goles del equipo local y la cantidad de empates en ese estadio, ocupando la información de las 6 temporadas, asumiendo que cada equipo siempre ha jugado de local en su estadio, eliminando los registros sin información y utilizando el 80% de los registros para entrenamiento, ¿cuál de las siguientes alternativas corresponde al código para calcular el coeficiente R2 de esta regresión redondeado a dos decimales? 

R: np.round(r2_score(y_test_p3, y_pred_p3),2)    



### 4

Pregunta 4

Asumiendo que la cantidad de goles que convertirá un equipo se predice como la cantidad de goles que ese equipo convirtió en la fecha justo anterior (sin importar si fue local o visita), ¿cuál de las siguientes alternativas corresponde al promedio del error absoluto entre la cantidad de goles totales y la cantidad de goles pronosticados para un partido, considerando todos los partidos desde la fecha 2 a la última de la temporada 21/22? 

R: 1.93

In [85]:
import pandas as pd

# Cargar los datos de todas las temporadas
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1617 = pd.read_excel('LaLiga16 17.xlsx')

# Concatenar todos los DataFrames en uno
seasons = [laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617]
df = pd.concat(seasons, ignore_index=True)
df = df[df['Wk'] >= 2]

# Ordenar por equipo y fecha
df.sort_values(by=['Local', 'Date'], inplace=True)

# Función para convertir el marcador a un entero
def convert_score_to_int(score):
    try:
        return int(score.split('–')[0])  # Tomar el primer valor antes del guion largo
    except (ValueError, AttributeError):
        return 0  # En caso de error, retornar 0

# Aplicar la conversión a la columna Score
df['Score'] = df['Score'].astype(str)
df['Score'] = df['Score'].apply(convert_score_to_int)

# Calcular los goles predichos como los goles del partido anterior
df['Goles_Predichos'] = df.groupby('Local')['Score'].shift(fill_value=0)

# Calcular el error absoluto
df['Error_Absoluto'] = abs(df['Score'] - df['Goles_Predichos'])

# Calcular el promedio del error absoluto con 2 decimales
promedio_error = df['Error_Absoluto'].mean()
promedio_error = round(promedio_error, 2)

promedio_error


1.28

In [86]:
import pandas as pd

# Cargar los datos de todas las temporadas
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1617 = pd.read_excel('LaLiga16 17.xlsx')

# Concatenar todos los DataFrames en uno
seasons = [laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617]
df = pd.concat(seasons, ignore_index=True)
df = df[df['Wk'] >= 2]
# Ordenar por equipo y fecha
df.sort_values(by=['Local', 'Date'], inplace=True)

# Función para convertir el marcador a un entero
def convert_score_to_int(score):
    try:
        return int(score.split('–')[0])  # Tomar el primer valor antes del guion largo
    except (ValueError, AttributeError):
        return 0  # En caso de error, retornar 0

# Aplicar la conversión a la columna Score
df['Score'] = df['Score'].astype(str)
df['Score'] = df['Score'].apply(convert_score_to_int)

# Agrupar por equipo y aplicar shift para obtener los goles predichos desde la fecha 2
df['Goles_Predichos'] = df.groupby('Local')['Score'].shift(fill_value=2)

# Calcular el error absoluto
df['Error_Absoluto'] = abs(df['Score'] - df['Goles_Predichos'])

# Calcular el promedio del error absoluto con 2 decimales
promedio_error = df['Error_Absoluto'].mean()
promedio_error = round(promedio_error, 2)

promedio_error


1.27

In [83]:
import pandas as pd

# Carga los datos desde los archivos Excel
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1617 = pd.read_excel('LaLiga16 17.xlsx')

# Concatena todos los DataFrames en uno solo
all_seasons = pd.concat([laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617])

# Ordena los datos por fecha para asegurarnos de que estén en el orden correcto
all_seasons.sort_values(by='Date', inplace=True)

# Filtra los partidos desde la fecha 2 hasta el final de la temporada 21/22
filtered_seasons = all_seasons[all_seasons['Wk'] >= 2]

# Convertir las columnas Score y Predicted_Goals a tipo numérico (int o float)
filtered_seasons['Score'] = pd.to_numeric(filtered_seasons['Score'], errors='coerce')
filtered_seasons['Predicted_Goals'] = pd.to_numeric(filtered_seasons['Predicted_Goals'], errors='coerce')

# Calcular el error absoluto
filtered_seasons['Absolute_Error'] = abs(filtered_seasons['Score'] - filtered_seasons['Predicted_Goals'])

# Calcular el promedio del error absoluto con 2 decimales
average_absolute_error = filtered_seasons['Absolute_Error'].mean()
average_absolute_error_rounded = round(average_absolute_error, 2)

print(f"Promedio del error absoluto: {average_absolute_error_rounded}")


KeyError: 'Predicted_Goals'

In [87]:
import pandas as pd

# Cargar los datos de todas las temporadas
laliga2122 = pd.read_excel('LaLiga21 22.xlsx')
laliga2021 = pd.read_excel('LaLiga20 21.xlsx')
laliga1920 = pd.read_excel('LaLiga19 20.xlsx')
laliga1819 = pd.read_excel('LaLiga18 19.xlsx')
laliga1718 = pd.read_excel('LaLiga17 18.xlsx')
laliga1617 = pd.read_excel('LaLiga16 17.xlsx')

# Concatenar todos los DataFrames en uno
seasons = [laliga2122, laliga2021, laliga1920, laliga1819, laliga1718, laliga1617]
df = pd.concat(seasons, ignore_index=True)
df = df[df['Wk'] >= 2]

# Ordenar por equipo y fecha
df.sort_values(by=['Local', 'Date'], inplace=True)

# Función para convertir el marcador a un entero
def convert_score_to_int(score):
    try:
        return int(score.split('–')[0])  # Tomar el primer valor antes del guion largo
    except (ValueError, AttributeError):
        return 0  # En caso de error, retornar 0

# Aplicar la conversión a la columna Score
df['Score'] = df['Score'].astype(str)
df['Score'] = df['Score'].apply(convert_score_to_int)

# Calcular los goles predichos como los goles del partido anterior por equipo
df['Goles_Predichos'] = df.groupby('Local')['Score'].shift(fill_value=0)

# Calcular el error absoluto
df['Error_Absoluto'] = abs(df['Score'] - df['Goles_Predichos'])

# Calcular el promedio del error absoluto con 2 decimales
promedio_error = df['Error_Absoluto'].mean()
promedio_error = round(promedio_error, 2)

promedio_error


1.26

### 5
Pregunta 5

Asuma que la cantidad de goles que convertirá un equipo se predice como la cantidad de goles que ese equipo convirtió en la fecha justo anterior (sin importar si fue local o visita). Utilice este pronóstico para predecir si el partido lo ganó el equipo Local, Vista o fue Empate. Defina como indicador de la calidad de esta predicción el acierto a este resultado con un 0 si se acertó el resultado y con un 1 si no. Bajo lo anterior, ¿cuál de las siguientes alternativas corresponde al promedio de aciertos de este método de predicción, considerando todos los partidos desde la fecha 2 a la última de la temporada 21/22? 

R: 0.34

### 6

Pregunta 6

Utilizando los resultados obtenidos en la Pregunta 5, ¿cuál de las siguientes alternativas corresponde a la cantidad de partidos que tuvieron como predicción del resultado un empate? ( Considere todos los resultados, incluida la semana 1)


R: 106